In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db = sqlite3.connect("Monarchs1994.db")

In [3]:
df = pd.read_csv("1994 Season Summary.csv")

        #convert pandas to SQL table
df.to_sql("monarchs1994", db, if_exists="replace")

os.getcwd()

C:\Users\Arink\Anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


'C:\\Users\\Arink\\Desktop\\github\\CodeLou\\Monarchs\\Data'

In [4]:
schema = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
schema

,type,name,tbl_name,rootpage,sql
0,index,ix_monarchs1994_index,monarchs1994,4,"CREATE INDEX ""ix_monarchs1994_index""ON ""monarc..."
1,table,monarchs1994,monarchs1994,2,"CREATE TABLE ""monarchs1994"" (\n""index"" INTEGER..."


## Checking DataType

In [5]:
data_type = pd.read_sql('SELECT * FROM PRAGMA_TABLE_INFO("monarchs1994")', db)
data_type

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Tag No.,TEXT,0,None,0
2,2,Tagger,TEXT,0,None,0
3,3,Mon Sex,TEXT,0,None,0
4,4,Location,TEXT,0,None,0
5,5,Date Tagged,TEXT,0,None,0
6,6,Date Recovered,TEXT,0,None,0
7,7,Location.1,TEXT,0,None,0
8,8,Observer,TEXT,0,None,0
9,9,Interval,TEXT,0,None,0


### Drop tagger and reporter identification

In [6]:
df.drop(["Tagger", "Mon Sex", "Observer", "Interval"], axis = 1, inplace = True)
df.head(10)

,Tag No.,Location,Date Tagged,Date Recovered,Location.1,Distance
0,145LA,"Tully, NY",NaN,29-Sep,"LaFayette, NY",8
1,134LK,Plattsburgh\nAir Force\nBase,19-Sep,25-Sep,Plattsburgh Air\nForce Base,0
2,945KX,"Kent Park, IA",10-Sep,29-Sep,"Mexico, MO",170
3,283LJ,"Carroll, IA",19-Sep,20-Sep,"Carroll, IA",0
4,237AQ,"Hays, KS",21-Sep,21-Sep,"Hays, KS",0
5,668KY,"DesMoines\n, IA",18-Sep,25-Sep,"North High\nSchool\nDes Moines, IA",0
6,873LT,"Shawnee,\nKS",15-Sep,18-Sep,"5mile east of\nDrexel, MO",40
7,824KF,"15m N.\nDecorah,\nIA",6-Sep,11-Sep,"Dyersville, IA",100
8,666KO,"Wichita,\nKS",5-Sep,6-Sep,"Wichita, KS",0
9,185KX,"Kansas\nCity, MO",NaN,27-Sep,"Kansas City,\nMO",0


### Rename Columns to match other years

In [7]:
df.rename(columns={'Distance':'Miles'}, inplace = True)

### Split & rename combined columns in Tagged category

In [8]:
df.loc[df['Location'].str.contains("\n", na=False), "Location"] = df.loc[df['Location'].str.contains("\n", na=False), "Location"].str.replace("\n", " ")
df2 = df['Location'].str.split(', ', expand=True, n=1)
df2.columns = ['tag_city', 'Tag State']
df2.loc[df2['Tag State'] == 'Quebec, Canada', 'Tag State'] = 'Quebec'
df2.loc[df2['Tag State'] == 'Coahuila, Mexico', 'Tag State'] = 'Coahuila'
df2.loc[df2['Tag State'].isnull(), 'Tag State'] = 'NY'
df2['Tag State'] = df2['Tag State'].str.strip(".")
df2.head(10)

,tag_city,Tag State
0,Tully,NY
1,Plattsburgh Air Force Base,NY
2,Kent Park,IA
3,Carroll,IA
4,Hays,KS
5,DesMoines,IA
6,Shawnee,KS
7,15m N. Decorah,IA
8,Wichita,KS
9,Kansas City,MO


### Split & Rename combined columns in Reported category

In [9]:
df.loc[df['Location.1'].str.contains("\n", na=False), "Location.1"] = df.loc[df['Location.1'].str.contains("\n", na=False), "Location.1"].str.replace("\n", " ")
df3 = df['Location.1'].str.split(',', expand=True, n=1)
df3.columns = ['report_city', 'Report State']
df3['Report State'] = df3['Report State'].str.strip()
df3['Report State'] = df3['Report State'].str.strip('.')
df3.loc[df3['Report State'] == 'Mexico', 'Report State'] = 'Michoacán'
df3.loc[df3['Report State'] == 'Essex Junction, VT', 'Report State'] = 'VT'
df3.loc[df3['Report State'] == 'Conn', 'Report State'] = 'CT'
df3.loc[df3['Report State'].isnull(), 'Report State'] = 'NY'
df3['Report State'] = df3['Report State'].str.replace(' ','')
df3.head(10)

,report_city,Report State
0,LaFayette,NY
1,Plattsburgh Air Force Base,NY
2,Mexico,MO
3,Carroll,IA
4,Hays,KS
5,North High School Des Moines,IA
6,5mile east of Drexel,MO
7,Dyersville,IA
8,Wichita,KS
9,Kansas City,MO


### Create new column Report Country & clean up

In [10]:
add_country = df['Location.1'].str.extract("(Angangueo)", expand=True)
add_country.columns = ['Report Country']
add_country.loc[add_country['Report Country'] == 'Angangueo', 'Report Country'] = 'MEXICO'
add_country.loc[add_country['Report Country'].isnull(), 'Report Country'] = 'USA'
add_country.head(10)

,Report Country
0,USA
1,USA
2,USA
3,USA
4,USA
5,USA
6,USA
7,USA
8,USA
9,USA


### Date columns, are in odd order, need to split then put back together 
#### Tag Date

In [11]:
fix_tag = df['Date Tagged'].str.extract('([a-zA-z]{3})', expand=True)
fix_tag.columns = ['tag_month']
fix_tag['tag_date'] = df['Date Tagged'].str.extract('(\d{1,2})', expand=True)
fix_tag['tag_year'] = '1994'
fix_tag['Tag Date'] = fix_tag["tag_month"] +" "+ fix_tag["tag_date"] +" "+ fix_tag["tag_year"]
#fix_tag.drop(['tag_month', 'tag_date', 'tag_year'], axis=1, inplace=True)
fix_tag.head(10)

,tag_month,tag_date,tag_year,Tag Date
0,NaN,NaN,1994,NaN
1,Sep,19,1994,Sep 19 1994
2,Sep,10,1994,Sep 10 1994
3,Sep,19,1994,Sep 19 1994
4,Sep,21,1994,Sep 21 1994
5,Sep,18,1994,Sep 18 1994
6,Sep,15,1994,Sep 15 1994
7,Sep,6,1994,Sep 6 1994
8,Sep,5,1994,Sep 5 1994
9,NaN,NaN,1994,NaN


#### Report Date

In [12]:
fix_rec = df['Date Recovered'].str.extract('([a-zA-z]{3})', expand=True)
fix_rec.columns = ['rec_month']
fix_rec['rec_date'] = df['Date Recovered'].str.extract('(\d{1,2})', expand=True)
fix_rec.loc[fix_rec['rec_date'] == '95', 'rec_date'] = ''
fix_rec.loc[(fix_rec['rec_month'] == 'Feb') | (fix_rec['rec_month'] == 'Mar'), 'rec_year']  = '1995' 
fix_rec['rec_year'].fillna('1994', inplace=True)
fix_rec['Report Date'] = fix_rec["rec_month"] +" "+ fix_rec["rec_date"] +" "+ fix_rec["rec_year"]
#fix_tag.drop(['tag_month', 'tag_date', 'tag_year'], axis=1, inplace=True)
fix_rec.head(10)

,rec_month,rec_date,rec_year,Report Date
0,Sep,29,1994,Sep 29 1994
1,Sep,25,1994,Sep 25 1994
2,Sep,29,1994,Sep 29 1994
3,Sep,20,1994,Sep 20 1994
4,Sep,21,1994,Sep 21 1994
5,Sep,25,1994,Sep 25 1994
6,Sep,18,1994,Sep 18 1994
7,Sep,11,1994,Sep 11 1994
8,Sep,6,1994,Sep 6 1994
9,Sep,27,1994,Sep 27 1994


### Create new table with unneeded columns dropped

In [13]:
df4 = pd.concat([df, df2, df3, add_country, fix_tag, fix_rec], axis=1)
df4.drop(['Location', 'Location.1', 'report_city', 'tag_city'], axis=1, inplace=True)
df5 = df4[['Tag No.','Tag State', 'Tag Date', 'Report State', 'Report Country', 'Report Date', 'Miles']]
new_table = df5.copy()
new_table

,Tag No.,Tag State,Tag Date,Report State,Report Country,Report Date,Miles
0,145LA,NY,NaN,NY,USA,Sep 29 1994,8
1,134LK,NY,Sep 19 1994,NY,USA,Sep 25 1994,0
2,945KX,IA,Sep 10 1994,MO,USA,Sep 29 1994,170
3,283LJ,IA,Sep 19 1994,IA,USA,Sep 20 1994,0
4,237AQ,KS,Sep 21 1994,KS,USA,Sep 21 1994,0
5,668KY,IA,Sep 18 1994,IA,USA,Sep 25 1994,0
6,873LT,KS,Sep 15 1994,MO,USA,Sep 18 1994,40
7,824KF,IA,Sep 6 1994,IA,USA,Sep 11 1994,100
8,666KO,KS,Sep 5 1994,KS,USA,Sep 6 1994,0
9,185KX,MO,NaN,MO,USA,Sep 27 1994,0


### Final check for unwanted spaces

In [14]:
# new_table.loc[new_table['Report State'].str.contains(r'\s', na=False), 'Report State'].value_counts()
# new_table.loc[new_table['Tag State'].str.contains(r'\s', na=False), 'Tag State'].value_counts()
# new_table.loc[new_table['Tag No.'].str.contains(r'\s', na=False), 'Tag No.'].value_counts()
# new_table.loc[new_table['Report Country'].str.contains(r'\s', na=False), 'Report Country'].value_counts()

### Convert date columns to DateTime datatypes

In [15]:
new_table["Tag Date"]= pd.to_datetime(new_table["Tag Date"]) 
new_table["Report Date"]= pd.to_datetime(new_table["Report Date"]) 
new_table.dtypes

Tag No.                   object
Tag State                 object
Tag Date          datetime64[ns]
Report State              object
Report Country            object
Report Date       datetime64[ns]
Miles                      int64
dtype: object

In [16]:
del df2, df3, df4, df5, add_country, fix_tag, fix_rec

### Export cleaned data to csv & pkl

In [17]:
new_table.to_pickle("Monarchs1994.pkl")
new_table.to_csv("Monarchs1994.csv")